In [1]:
#Naive Bayes Classifier for Iris Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv('../Assignment_data/Data_Q2/iris.csv') #Loading the data from iris dataset
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [2]:
#Split the data into test and train data
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

In [3]:
%%time
import math
from numpy.linalg import pinv
from sklearn.preprocessing import Imputer

#train_set.head()
train_data = train_set.copy() #all the data set.
train_data["species"] = ((train_data["species"]=="virginica").astype(int)) #Converting the string labels to int labels and having two classes.


def summary(train_data): #To find the mean and covariance matrix of both the classes.
    #separate the data by classes
    class1_data = (train_data.loc[train_data['species']==0]).drop(["species"],axis=1)
    class2_data = (train_data.loc[train_data['species']==1]).drop(["species"],axis=1)

    mean_class1 = np.array(class1_data.mean())
    mean_class2 = np.array(class2_data.mean())
    cov_class1 = np.array(class1_data.cov())
    cov_class2 = np.array(class2_data.cov())

    mean = []
    mean.append(mean_class1)
    mean.append(mean_class2)
    cova = []
    cova.append(cov_class1)
    cova.append(cov_class2)
    
    return mean,cova

mean,cova = summary(train_data) #mean and covariance matrix of the two classes in train_data

def Gaussian(x,mean,cova):#Probability of a given class i.e P(x/c) = 1/(((2*pi)^d/2*det(co-matrix)^1/2) * e^-((x-mean).T.inv(co-matrix).(x-mean))/2)
    d = len(x)
    exponent = math.exp((-1)*(((x-mean).T).dot(pinv(cova)).dot((x-mean))/2.0))
    result = (exponent)/(math.sqrt(((2*math.pi)**d)*np.linalg.det(cova)))
    return result

CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 633 ms


In [4]:
def predictLabel(mean,cova,x): #Prediction of the class label based on the probabilty of the class
    if(Gaussian(x,mean[0],cova[0]) > Gaussian(x,mean[1],cova[1])):
        return 0
    return 1

def getPredictions(mean,cova,data): #To get the Prected labels for whole test data
    predictions = []
    for i in range(len(data)):
        result = predictLabel(mean,cova,data[i])
        predictions.append(result)
    return np.array(predictions)

test_data = test_set.drop(["species"],axis=1)
test_labels = np.array((test_set["species"]=="virginica").astype(int))
predictedLabels = getPredictions(mean,cova,np.array(test_data))
predictedLabels

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 0])

In [5]:
from sklearn.metrics import accuracy_score #Calculating Accuracy
accuracy_score(test_labels,predictedLabels)

0.9666666666666667